In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

21583

In [3]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [4]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}&page={i}')

100%|███████████████████████████████████| 21583/21583 [00:18<00:00, 1151.01it/s]


In [5]:
urls = sorted(list(set(urls)))
len(urls)

343

In [6]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    td = soup.find_all('td', {'class': 't_f'})
    td = [str(t) for t in td]
    return {'td': td, 'url': url}

In [7]:
# !rm -rf pages
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [8]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|██████████████████████████████████████████| 7/7 [00:00<00:00, 43050.04it/s]


In [9]:
import pandas as pd

In [29]:
files = glob('threads/*.json')

df = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        forum_id = d[0].split('tid=')[1]
        for d_ in d[1]:
            soup = BeautifulSoup(d_, "lxml")
            id = soup.find('td', {'class': 't_f'}).get('id').split('_')[1]
            quote = soup.find('div', {'class':"quote"})
            if quote is not None:
                quote_text = str(quote.find('blockquote')).split('<br/>')[0].replace('<blockquote>', '').replace('...', '').strip()
                soup.find('div', {'class':"quote"}).decompose()
            else:
                quote_text = None
            
            text = soup.get_text(separator="\n")
            df.append({
                'forum_id': forum_id,
                'id': id,
                'text': text,
                'quote': quote_text,
                'quote_id': None,
            })

100%|█████████████████████████████████████| 21583/21583 [19:09<00:00, 18.77it/s]


In [30]:
files = glob('pages/*.json')

for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        forum_id = d['url'].split('tid=')[1].split('&')[0]
        for d_ in d['td']:
            soup = BeautifulSoup(d_, "lxml")
            id = soup.find('td', {'class': 't_f'}).get('id').split('_')[1]
            quote = soup.find('div', {'class':"quote"})
            if quote is not None:
                quote_text = str(quote.find('blockquote')).split('<br/>')[0].replace('<blockquote>', '').replace('...', '').strip()
                soup.find('div', {'class':"quote"}).decompose()
            else:
                quote_text = None
            
            text = soup.get_text(separator="\n")
            df.append({
                'forum_id': forum_id,
                'id': id,
                'text': text,
                'quote': quote_text,
                'quote_id': None,
            })

100%|█████████████████████████████████████████████| 7/7 [00:02<00:00,  2.95it/s]


In [31]:
df_pd = pd.DataFrame(df)

In [32]:
df_pd.to_parquet('posts.parquet', index = False)

In [14]:
# dfs = []
# for forum_id in tqdm(df_pd['forum_id'].unique().tolist()):
#     df_pd_ = df_pd[df_pd['forum_id'] == forum_id]
#     for i in range(len(df_pd_)):
#         for k in range(len(df_pd_)):
#             if df_pd_['quote'].iloc[i] is not None and df_pd_['quote'].iloc[i] in df_pd_['text'].iloc[k].replace('\n', ''):
#                 df_pd_['quote_id'].iloc[i] = df_pd_['id'].iloc[k]
#                 break
#     dfs.append(df_pd_)

  0%|                                                  | 0/1506 [00:00<?, ?it/s]/home/husein/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|███████████████████████████████████████| 1506/1506 [01:09<00:00, 21.79it/s]


In [15]:
# texts = list(set(texts))
# len(texts)

In [ ]:
# with open('everything.jsonl', 'w') as fopen:
#     for t in tqdm(texts):
#         if len(t):
#             fopen.write(f'{json.dumps(t)}\n')

In [ ]:
# !zip -r threads.zip threads
# !zip -r pages.zip pages
# !zip -r forums.zip forums